In [5]:
# Import Libraries
import geopandas as gpd
import fiona
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import subprocess
import time

In [6]:
# Define the GeoProcessor class
class GeoProcessor:
    # Initialize the class
    def __init__(self):
        # Set the default shapefiles to None
        self.shapefile1 = None
        # self.shapefile2 = None
        self.shapefile2 = None
        # Set the default output folder to "output"
        self.output_folder = "output"
    # Load shapefiles
    def load_shapefiles(self, shapefile1, shapefile2):
        # Check if the shapefiles exist
        if os.path.exists(shapefile1) and os.path.exists(shapefile2):
            # Set the shapefiles as class attributes
            self.shapefile1 = shapefile1
            # self.shapefile2 = shapefile2
            self.shapefile2 = shapefile2
            # Return success message
            return True, "Shapefiles loaded successfully!"
        else:
            # Return error message
            return False, "Please provide valid shapefile paths."
    # Process intersection
    def process_intersection(self):
        # Check if shapefiles are loaded
        if self.shapefile1 and self.shapefile2:
            # Start timing
            try:
                # Start timing
                start_time = time.time() 
                # Check if SHAPE_RESTORE_SHX is set
                with fiona.Env(SHAPE_RESTORE_SHX=True):
                    # Read shapefiles 1
                    gdf1 = gpd.read_file(self.shapefile1)
                    # Read shapefiles 2
                    gdf2 = gpd.read_file(self.shapefile2)
                # Perform intersection
                intersection_gdf = gpd.overlay(gdf1, gdf2, how='intersection')
                # Save intersection results
                if not os.path.exists(self.output_folder):
                    # Create output folder if it doesn't exist
                    os.makedirs(self.output_folder)
                    # Save intersection results
                intersection_file = os.path.join(self.output_folder, "intersection_results.shp")
                # Save intersection results 
                intersection_gdf.to_file(intersection_file)
                # End timing
                end_time = time.time() 
                # Calculate elapsed time
                elapsed_time = end_time - start_time
                # Return success message
                return len(intersection_gdf), f"Intersection results saved to '{intersection_file}'. Process took {elapsed_time:.2f} seconds."
                # Return error message
            except Exception as e:
                return 0, f"Failed to process intersection: {e}"
        else:
            return 0, "Shapefiles not loaded."

In [7]:
# Define the GeoGUI class
class GeoGUI:
    # Initialize the class
    def __init__(self, root):
        # Initialize the GeoProcessor class
        self.processor = GeoProcessor()
        # Set the root window
        self.root = root
        # Set the window title and size
        self.root.title("Polygon Intersection Processor")
        # Set the window geometry
        self.root.geometry("600x400")
        # Disable resizing the window
        self.root.resizable(False, False)

        # Create the main frame
        self.main_frame = ttk.Frame(root)
        # Pack the main frame
        self.main_frame.pack(pady=20)
        # Create the widgets
        self.shapefile1_label = ttk.Label(self.main_frame, text="Shapefile 1:")
        # Grid the widgets
        self.shapefile1_label.grid(row=0, column=0, padx=5, pady=5, sticky="e")
        # Create the entry widget
        self.shapefile1_path = ttk.Entry(self.main_frame, width=40)
        # Grid the entry widget
        self.shapefile1_path.grid(row=0, column=1, padx=5, pady=5)
        # create the browse button
        self.shapefile1_button = ttk.Button(self.main_frame, text="Browse", command=self.browse_shapefile1)
        self.shapefile1_button.grid(row=0, column=2, padx=5, pady=5)

        # Create the shapefile 2 label and entry widget
        self.shapefile2_label = ttk.Label(self.main_frame, text="Shapefile 2:")
        self.shapefile2_label.grid(row=1, column=0, padx=5, pady=5, sticky="e")
        self.shapefile2_path = ttk.Entry(self.main_frame, width=40)
        self.shapefile2_path.grid(row=1, column=1, padx=5, pady=5)
        self.shapefile2_button = ttk.Button(self.main_frame, text="Browse", command=self.browse_shapefile2)
        self.shapefile2_button.grid(row=1, column=2, padx=5, pady=5)
        # Create the process button
        self.process_button = ttk.Button(self.main_frame, text="Process Intersection", command=self.process_intersection)
        self.process_button.grid(row=2, column=0, columnspan=3, pady=20)

        # Create the results label and text widget
        self.results_label = ttk.Label(self.main_frame, text="Results:")
        self.results_label.grid(row=3, column=0, columnspan=3, sticky="w")
        self.results_text = tk.Text(self.main_frame, height=10, width=60, wrap="word", bg="#f0f0f0", state="disabled")
        self.results_text.grid(row=4, column=0, columnspan=3, pady=10)

        # Create the open folder button
        self.open_folder_button = ttk.Button(self.main_frame, text="Open Results Folder", command=self.open_results_folder)
        self.open_folder_button.grid(row=5, column=0, columnspan=3, pady=10)
    # Create Browse Shapefile 1 button
    def browse_shapefile1(self):
        file_path = filedialog.askopenfilename(filetypes=[("Shapefiles", "*.shp")])
        if file_path:
            self.shapefile1_path.delete(0, tk.END)
            self.shapefile1_path.insert(0, file_path)

    # Create Browse Shapefile 2 button
    def browse_shapefile2(self):
        file_path = filedialog.askopenfilename(filetypes=[("Shapefiles", "*.shp")])
        if file_path:
            self.shapefile2_path.delete(0, tk.END)
            self.shapefile2_path.insert(0, file_path)
    # Create Process Intersection button
    def process_intersection(self):
        # Load shapefiles
        shapefile1 = self.shapefile1_path.get()
        shapefile2 = self.shapefile2_path.get()
        # Process intersection
        success, message = self.processor.load_shapefiles(shapefile1, shapefile2)
        # Process intersection
        if success:
            # Process intersection
            num, result_message = self.processor.process_intersection()
            # Display results
            self.results_text.config(state="normal")
            # Clear previous results
            self.results_text.delete(1.0, tk.END)
            # Insert results
            self.results_text.insert(tk.END, f"Number of intersected polygons: {num}\n")
            self.results_text.insert(tk.END, result_message)
            self.results_text.config(state="disabled")
        else:
            messagebox.showerror("Error", message)
    # Create Open Results Folder button
    def open_results_folder(self):
        # Open the output folder in the default file explorer
        subprocess.Popen(["explorer", self.processor.output_folder])

In [9]:
# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = GeoGUI(root)
    root.mainloop()